In [1]:
import json
from typing import List, Tuple, Sequence
import pickle
from tqdm import tqdm_notebook as tqdm
import pymorphy2
import nltk
from cytoolz import pipe
from collections import Counter
import re
from collections import defaultdict
from collections import Counter

In [2]:
morph = pymorphy2.MorphAnalyzer()
stopwords = nltk.corpus.stopwords.words(
    'russian') + nltk.corpus.stopwords.words('english')
stopwords += [
    'отличный', 'метр', 'наш', 'клиент', 'банка', 'проект', 'литр',
    'желательный', 'др', 'самый', 'мочь', 'хороший', 'год', 'чел', 'обязательный'
]

cache = {}


def remove_numbers(text):
    return re.sub(r'\d+', '', text)


def _get_POS(word):
    if word not in cache:
        cache[word] = morph.parse(word)[0]
    return cache[word].tag.POS


def normal_form(word):
    if word not in cache:
        cache[word] = morph.parse(word)[0]
    return cache[word].normal_form


def is_word_pos_in(word: str, pos: List[str] = None) -> bool:
    if not pos:
        pos = ['NOUN', "ADJF", 'INFN', 'VERB', 'ADJS']

    return _get_POS(word) in pos


def get_words(text):
    return re.findall(r'\w+', text)


def nonempty(x):
    if isinstance(x, Sequence):
        return filter(lambda x: len(x) > 0 and x != ' ', x)
    return x

helper = {}


def remove_numbers(text):
    return re.sub(r'\d+', '', text)


def normalize_skill(skill: str):
    parsed = tuple(
        pipe(
            skill,
            lambda x: x.lower(),
            remove_numbers,
            get_words,
        ))
    
    clear_skill = []
    dirty_skill = []
    
    # Последнее стоп слово для dirty_skill
    last_stopword = None
    
    # Для каждого слова в скилле
    for i in parsed:
        # Нормализуем слово
        word = normal_form(i)
        
        # Если стоп слово - запомним его
        if word in nltk.corpus.stopwords.words('russian'):
            last_stopword = word
            
            if word == "без":
                clear_skill.append(word)
        
        # Проверим на часть речи, длинну и стоплова
        elif is_word_pos_in(word) and len(word) > 3 and word not in stopwords:
            
            # Если до этого было стоп слово, добавим его в dirty
            if last_stopword and len(dirty_skill) > 0:
                dirty_skill.append(last_stopword)
                last_stopword = None
            
            # Добавим в чистый скилл слово
            clear_skill.append(word)
            
            if is_word_pos_in(word, ['NOUN', 'ADJF']):
                dirty_skill.append(i)

    if len(clear_skill) > 1 and len(clear_skill) < 8:
        return clear_skill, dirty_skill
    
    else:
        return []

In [3]:
def split_skill(a):
    '''Рассказ о программистах или о жизни -> рассказ о программистах, рассказ о жизни'''
    main_skill = a

    def repl(x): return x.replace(',', '|').replace(
        ' или ', '|').replace(' и ', '|').replace('/', '|').split('|')

    first_match = re.match(r'([а-яА-ЯA-Za-z\-\s]*([,и\/]|или)\s)', a)

    if first_match:
        first_match = first_match.group()
        a = repl(a.replace(first_match, ''))

        for i in a:
            if len(i.split()) > 4:
                return repl(main_skill)

        variants = first_match.replace(',', ' ').replace(
            ' или ', ' ').replace(' и ', ' ').replace('/', ' ')
        variants = list(filter(len, variants.split(' ')))
        if len(variants) > 1:
            main_phrase = variants[:-1]
            a.append(variants[-1])

            skills = []

            for i in a:
                skills.append(" ".join(main_phrase + [i]))
            return skills
        else:
            return a + variants
    else:
        return [a]

In [4]:
def split_into_skills(text: str) -> List[str]:
    def _split_into_skills(x):
        x = re.sub(r'([А-ЯA-Z])', r'\n\1', text)
        return re.split(r'[\n\.,]', x)

    pre_skills = list(nonempty(_split_into_skills(text)))
    done_skills = []

    for skill in pre_skills:
        skill = skill.replace('/', ' / ').replace("\xa0", " ")
        if len(re.findall("\([а-яА-ЯA-Za-z\-\s]*\)", skill)) > 0:
            skill1 = re.findall(r'\([а-яА-ЯA-Za-z\-\s]*\)', skill)[0]
            skill2 = re.findall(r'[а-яА-ЯA-Za-z\-\s]*\(', skill)[0][:-1]
            done_skills.extend([skill1, skill2])
        elif len(re.findall("([а-яА-ЯA-Za-z\-\s]*([и\/]|или)\s[а-яА-ЯA-Za-z\-])", skill)) > 0:
            new_skills = split_skill(skill)
            if new_skills:
                done_skills.extend(new_skills)

        else:
            done_skills.append(skill)
    return done_skills

In [5]:
from joblib import delayed, Parallel

def parallel(f, data):
    """Run parallel your func on all CPU"""
    
    return Parallel(n_jobs=-1, verbose=3, max_nbytes='1G')(delayed(f)(x) for x in data)

In [6]:
def load_resume_vacancy() -> Tuple[List[dict], List[dict]]:
    """Loads JSON data of SuperJob"""

    def load_json_by_lines(file):
        tmp = []
        
        with open(file) as f:
            for line in tqdm(f.read().split("\n")):
                if len(line) > 0:
                    tmp.append(json.loads(line))
        return tmp
    
    resume = load_json_by_lines("data/resume.json")
    vacancy = load_json_by_lines("data/vacancy.json")
    
    return resume, vacancy

In [ ]:
resume, vacancy = load_resume_vacancy()

In [ ]:
pickle.dump(resume, open("resume.pck", "wb"))
pickle.dump(vacancy, open("vacancy.pck", "wb"))

In [7]:
resume = pickle.load(open("resume.pck", "rb"))
vacancy = pickle.load(open("vacancy.pck", "rb"))

In [ ]:
def add_skills(vac):
    vac = vac.copy()
    skills = split_into_skills(vac['candidat'])

    clear_skills, dirty_skills = [], []
    
    for skill in skills:
        normalized = normalize_skill(skill)
        
        if len(normalized) > 0:
            clear_skills.append(" ".join(normalized[0]))
            dirty_skills.append(" ".join(normalized[1]))
        
    vac['clear_skills'] = clear_skills
    vac['dirty_skills'] = dirty_skills
    
    return vac

In [ ]:
data = []

for vac in tqdm(vacancy):
    data.append(add_skills(vac))

In [ ]:
#pickle.dump(data, open("vacancy_with_skills.pck", "wb"))

In [8]:
data = pickle.load(open("vacancy_with_skills.pck", "rb"))

In [9]:
themes = defaultdict(lambda: [])
dirty_to_normal = {}

for d in data:
    themes[d['profession_tree_name']].extend(d['dirty_skills'])

In [10]:
works = []

for i in resume:
    works.extend(i['work_history'])

In [ ]:
# def f(work):      
#     print(work)
#     clear_skills, dirty_skills = [], []
    
#     for skill in split_into_skills(work['work']):
#         normalized = normalize_skill(skill)
        
#         if len(normalized) > 0:
#             clear_skills.append(" ".join(normalized[0]))
#             dirty_skills.append(" ".join(normalized[1]))
        
#     return {"name": work['name'], "clear_skills": clear_skills, "dirty_skills": dirty_skills} 

In [ ]:
# works_clear = parallel(f, works)

In [ ]:
# def f(i):
#     tmp = []
#     for j in split_into_skills(i['work']):
#         for g in get_words(j.lower()):
#             tmp.append(g)
#     return tmp
# words = parallel(f, works)

In [ ]:
# words_only = []
# for word in words:
#     for k in word:
#         words_only.append(k)

In [ ]:
# words_only = list(set(words_only))

In [ ]:
# cache = {}

# for word in tqdm(words_only):
#     cache[word] = morph.parse(word)[0]

In [ ]:
# get_words(works[0]["work"])

In [ ]:
# for work in tqdm(works):
#     text = work["work"]
#     text = text.lower()
#     text = remove_numbers(text)
#     text = split_into_skills(text)
    
#     for i in range(len(text)):
#         text[i] = get_words(text[i])
        
#         def g(x):
#             if x in cache:
#                 return cache[x].normal_form
#             else:
#                 print(x)
#                 return None
#         text[i] = list(filter(lambda x: x, map(g, text[i])))
        
#     work["work_clear"] = text